In [ ]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

In [ ]:
# ===== User Configurable Parameters =====
root_data_dir = "E:/GitHub/360videoplayer/Assets/TrailData"   # Root data directory
target_video  = "20180717_monkey cinema_focal manny_injected_360"
time_tolerance = 0.5   # seconds

# ===== POI Settings =====
enable_poi = True  # Set to True to show POI markers, False to hide them

# ===== Data Reading Mode Configuration =====
# Mode 1: Read all data for a single user
# mode = "single_user"
# user_name = "Control_6"

# Mode 2: Read all group data (generates 4 charts: Guidance first, Control first, Guidance all, Control all)
mode = "all_groups_enhanced"

# Mode 3: Read first session data for all users
# mode = "first_session_all"

# Current selected mode
print(f"Current data reading mode: {mode}")
if mode == "single_user":
    print(f"Analyzing user: {user_name}")
elif mode == "all_groups":
    print("Analyzing all groups: Guidance + Control")
elif mode == "all_groups_enhanced":
    print("Enhanced mode: Generate 4 charts - Guidance first, Control first, Guidance all data, Control all data")

print(f"POI markers: {'Enabled' if enable_poi else 'Disabled'}")

In [ ]:
def normalize_horiz_angle(a: int) -> int:
    """Normalize horizontal angle to -180 to 180 degree range"""
    return ((a + 180) % 360) - 180

def load_poi_data():
    """Load POI data"""
    poi_file = os.path.join(os.path.dirname(root_data_dir), "poi.json")
    print(f"Attempting to read POI file: {poi_file}")
    try:
        with open(poi_file, 'r', encoding='utf-8') as f:
            poi_data = json.load(f)
        print(f"Successfully loaded POI data: {len(poi_data['points'])} points of interest")
        return poi_data['points']
    except Exception as e:
        print(f"Failed to read POI data: {e}")
        # Try alternative path
        alt_poi_file = "E:/GitHub/360videoplayer/Assets/poi.json"
        print(f"Trying alternative path: {alt_poi_file}")
        try:
            with open(alt_poi_file, 'r', encoding='utf-8') as f:
                poi_data = json.load(f)
            print(f"Successfully loaded POI data: {len(poi_data['points'])} points of interest")
            return poi_data['points']
        except Exception as e2:
            print(f"Alternative path also failed: {e2}")
            return []

def get_session_directories():
    """Get session directories to process based on mode"""
    session_dirs = []
    guidance_dirs = []
    control_dirs = []
    guidance_first_dirs = []
    control_first_dirs = []
    
    if mode == "single_user":
        # Mode 1: Read all data for a single user
        user_dir = os.path.join(root_data_dir, user_name)
        if os.path.exists(user_dir):
            for d in os.listdir(user_dir):
                if target_video in d and not d.endswith('.meta'):
                    session_dirs.append(os.path.join(user_name, d))
        title = f"{user_name} — All Sessions"
        return session_dirs, title, [], [], [], []
        
    elif mode == "all_groups":
        # Mode 2: Process Guidance and Control group data separately
        for user_dir in os.listdir(root_data_dir):
            if not user_dir.endswith('.meta'):
                user_path = os.path.join(root_data_dir, user_dir)
                if os.path.isdir(user_path):
                    for d in os.listdir(user_path):
                        if target_video in d and not d.endswith('.meta'):
                            session_path = os.path.join(user_dir, d)
                            session_dirs.append(session_path)
                            
                            # Classify into different groups
                            if user_dir.startswith("Guidance_"):
                                guidance_dirs.append(session_path)
                            elif user_dir.startswith("Control_"):
                                control_dirs.append(session_path)
        
        print(f"Guidance group sessions: {len(guidance_dirs)}")
        print(f"Control group sessions: {len(control_dirs)}")
        return session_dirs, "Group Comparison", guidance_dirs, control_dirs, [], []
        
    elif mode == "all_groups_enhanced":
        # Enhanced mode: Process Guidance and Control group data separately, including first sessions and all data
        users_first_sessions = {}
        
        for user_dir in os.listdir(root_data_dir):
            if not user_dir.endswith('.meta'):
                user_path = os.path.join(root_data_dir, user_dir)
                if os.path.isdir(user_path):
                    user_sessions = []
                    # Collect all sessions for this user
                    for d in os.listdir(user_path):
                        if target_video in d and not d.endswith('.meta'):
                            session_path = os.path.join(user_dir, d)
                            session_dirs.append(session_path)
                            
                            # Classify into different groups (all data)
                            if user_dir.startswith("Guidance_"):
                                guidance_dirs.append(session_path)
                            elif user_dir.startswith("Control_"):
                                control_dirs.append(session_path)
                            
                            # Extract timestamp to find first session data
                            parts = d.split('_')
                            if len(parts) >= 3:
                                timestamp = parts[2]  # Timestamp part
                                user_sessions.append((timestamp, session_path))
                    
                    # Sort by timestamp, take the first as first session data
                    if user_sessions:
                        user_sessions.sort()
                        first_session_path = user_sessions[0][1]
                        users_first_sessions[user_dir] = first_session_path
                        
                        # Classify first session data
                        if user_dir.startswith("Guidance_"):
                            guidance_first_dirs.append(first_session_path)
                        elif user_dir.startswith("Control_"):
                            control_first_dirs.append(first_session_path)
        
        print(f"Guidance group all sessions: {len(guidance_dirs)}")
        print(f"Control group all sessions: {len(control_dirs)}")
        print(f"Guidance group first sessions: {len(guidance_first_dirs)}")
        print(f"Control group first sessions: {len(control_first_dirs)}")
        
        print(f"\nFound user first sessions:")
        for user, session in users_first_sessions.items():
            print(f"  {user}: {session}")
            
        return session_dirs, "Enhanced Group Comparison", guidance_dirs, control_dirs, guidance_first_dirs, control_first_dirs
        
    elif mode == "first_session_all":
        # Mode 3: Read first session data for all users
        users_first_sessions = {}
        
        for user_dir in os.listdir(root_data_dir):
            if not user_dir.endswith('.meta'):
                user_path = os.path.join(root_data_dir, user_dir)
                if os.path.isdir(user_path):
                    user_sessions = []
                    for d in os.listdir(user_path):
                        if target_video in d and not d.endswith('.meta'):
                            # Extract timestamp for sorting
                            parts = d.split('_')
                            if len(parts) >= 3:
                                timestamp = parts[2]  # Timestamp part
                                user_sessions.append((timestamp, d))
                    
                    # Sort by timestamp, take the first
                    if user_sessions:
                        user_sessions.sort()
                        first_session = user_sessions[0][1]
                        session_dirs.append(os.path.join(user_dir, first_session))
                        users_first_sessions[user_dir] = first_session
        
        title = "All Users — First Sessions"
        print(f"Found user first sessions:")
        for user, session in users_first_sessions.items():
            print(f"  {user}: {session}")
        return session_dirs, title, [], [], [], []

def process_group_data(group_dirs, group_name):
    """Process data for specified group, return heatmap data"""
    print(f"\n=== Processing {group_name} Group Data ===")
    
    # Initialize accumulators
    H_MIN, H_MAX = -180, 180
    V_MIN, V_MAX = -90, 90
    h_cols = H_MAX - H_MIN + 1
    v_cols = V_MAX - V_MIN + 1
    
    heat_h_tot = [[0]*h_cols for _ in range(total_sec)]
    heat_v_tot = [[0]*v_cols for _ in range(total_sec)]
    
    processed_sessions = 0
    
    for sess in group_dirs:
        session_path = os.path.join(root_data_dir, sess)
        horizontal_file = os.path.join(session_path, "Horizontal.json")
        vertical_file = os.path.join(session_path, "Vertical.json")
        
        if not (os.path.exists(horizontal_file) and os.path.exists(vertical_file)):
            print(f"Warning: Skipping {sess}, missing JSON files")
            continue
        
        try:
            hj = json.load(open(horizontal_file))
            vj = json.load(open(vertical_file))
            
            pts_h = sorted(hj["points"], key=lambda x: x["time"])
            pts_v = sorted(vj["points"], key=lambda x: x["time"])
            half_h = hj.get("fov", 60) / 2.0
            half_v = vj.get("fov", 60) / 2.0

            # Match time points
            i = j = 0
            matched = []
            while i < len(pts_h) and j < len(pts_v):
                th, tv = pts_h[i]["time"], pts_v[j]["time"]
                if abs(th - tv) <= time_tolerance:
                    matched.append(((th+tv)/2, pts_h[i]["angle"], pts_v[j]["angle"]))
                    i += 1; j += 1
                elif th < tv:
                    i += 1
                else:
                    j += 1

            # Accumulate to heatmap
            for t, hc, vc in matched:
                ti = int(t)
                if ti >= total_sec:
                    continue
                    
                hc0 = int(round(hc)); vc0 = int(round(vc))

                # Horizontal angle FOV
                for a in range(int(round(hc0-half_h)), int(round(hc0+half_h))+1):
                    an = normalize_horiz_angle(a)
                    ci = an - H_MIN
                    if 0 <= ci < h_cols:
                        heat_h_tot[ti][ci] += 1

                # Vertical angle FOV
                for a in range(max(V_MIN, int(round(vc0-half_v))),
                               min(V_MAX, int(round(vc0+half_v)))+1):
                    ci = a - V_MIN
                    if 0 <= ci < v_cols:
                        heat_v_tot[ti][ci] += 1
            
            processed_sessions += 1
            print(f"Processing completed: {sess} ({len(matched)} matched points)")
            
        except Exception as e:
            print(f"Error: Exception occurred while processing {sess}: {e}")
    
    print(f"{group_name} group successfully processed {processed_sessions} sessions")
    
    # Normalization
    def normalize_rows(mat):
        norm = []
        for row in mat:
            m = max(row) if row else 0
            if m == 0:
                norm.append(row)
            else:
                norm.append([v / m for v in row])
        return norm

    heat_h_norm = normalize_rows(heat_h_tot)
    heat_v_norm = normalize_rows(heat_v_tot)
    
    # Generate hover text
    text_h = [["" for _ in range(h_cols)] for _ in range(total_sec)]
    text_v = [["" for _ in range(v_cols)] for _ in range(total_sec)]

    for ti in range(total_sec):
        for ci in range(h_cols):
            cnt = heat_h_tot[ti][ci]
            if cnt:
                angle = H_MIN + ci
                text_h[ti][ci] = f"Time: {ti//60}:{ti%60:02d} | Horizontal: {angle}° | Count: {cnt}"
        for ci in range(v_cols):
            cnt = heat_v_tot[ti][ci]
            if cnt:
                angle = V_MIN + ci
                text_v[ti][ci] = f"Time: {ti//60}:{ti%60:02d} | Vertical: {angle}° | Count: {cnt}"
    
    return heat_h_norm, heat_v_norm, text_h, text_v, processed_sessions

def create_heatmap_with_poi(heat_h_norm, heat_v_norm, text_h, text_v, group_name, sessions_count, poi_points):
    """Create heatmap with POI markers"""
    H_MIN, H_MAX = -180, 180
    V_MIN, V_MAX = -90, 90
    
    fig = make_subplots(
        rows=2, cols=3,
        specs=[[ None, {"type":"heatmap"}, None ],
               [{"colspan":3,"type":"heatmap"}, None, None]],
        column_widths=[0.1, 0.8, 0.1],
        row_heights=[0.85, 0.15],
        vertical_spacing=0.02
    )

    # Top: Horizontal angle-time heatmap
    fig.add_trace(go.Heatmap(
        z=heat_h_norm,
        x=list(range(H_MIN, H_MAX + 1)),
        y=list(range(total_sec)),
        text=text_h,
        hoverinfo="text",
        colorscale="Viridis",
        zmin=0, zmax=1,
        showscale=True,
        name="Eye Tracking Heatmap"
    ), row=1, col=2)

    # Bottom: Vertical angle-time heatmap
    fig.add_trace(go.Heatmap(
        z=[list(col) for col in zip(*heat_v_norm)],
        x=list(range(total_sec)),
        y=list(range(V_MIN, V_MAX + 1)),
        text=[list(col) for col in zip(*text_v)],
        hoverinfo="text",
        colorscale="Viridis",
        zmin=0, zmax=1,
        showscale=False,
        name="Eye Tracking Heatmap"
    ), row=2, col=1)
    
    # ===== Add POI red dot markers =====
    if enable_poi and poi_points:
        print(f"Adding {len(poi_points)} POI markers to heatmap")
        
        # Add POI points on horizontal angle-time chart
        poi_times_h = []
        poi_hangles = []
        poi_hints_h = []
        
        for poi in poi_points:
            if poi['time'] < total_sec:
                poi_times_h.append(poi['time'])
                poi_hangles.append(poi['Hangle'])
                poi_hints_h.append(f"<b>POI Point of Interest</b><br>Time: {poi['time']}s<br>Horizontal: {poi['Hangle']}°<br>Vertical: {poi['Vangle']}°<br>Hint: {poi['hint']}")
        
        if poi_times_h:
            fig.add_trace(go.Scatter(
                x=poi_hangles,
                y=poi_times_h,
                mode='markers',
                marker=dict(
                    color='red',
                    size=12,
                    symbol='circle',
                    line=dict(color='white', width=2)
                ),
                name='POI Points',
                text=poi_hints_h,
                hoverinfo='text',
                showlegend=True
            ), row=1, col=2)
        
        # Add POI points on vertical angle-time chart
        poi_times_v = []
        poi_vangles = []
        poi_hints_v = []
        
        for poi in poi_points:
            if poi['time'] < total_sec:
                poi_times_v.append(poi['time'])
                poi_vangles.append(poi['Vangle'])
                poi_hints_v.append(f"<b>POI Point of Interest</b><br>Time: {poi['time']}s<br>Horizontal: {poi['Hangle']}°<br>Vertical: {poi['Vangle']}°<br>Hint: {poi['hint']}")
        
        if poi_times_v:
            fig.add_trace(go.Scatter(
                x=poi_times_v,
                y=poi_vangles,
                mode='markers',
                marker=dict(
                    color='red',
                    size=12,
                    symbol='circle',
                    line=dict(color='white', width=2)
                ),
                name='POI Points',
                text=poi_hints_v,
                hoverinfo='text',
                showlegend=False  # Avoid duplicate legend
            ), row=2, col=1)
    elif enable_poi:
        print("POI enabled but no POI data found, showing heatmap only")

    # Axis settings
    fig.update_xaxes(title_text="Horizontal Angle (°)", range=[H_MIN, H_MAX], row=1, col=2)
    fig.update_yaxes(
        title_text="Time (mm:ss)",
        autorange=False,
        range=[0, total_sec-1],
        tickmode="array",
        tickvals=tickvals,
        ticktext=ticktext,
        showticklabels=True,
        row=1, col=2
    )

    fig.update_xaxes(
        title_text="Time (mm:ss)",
        range=[0, total_sec-1],
        tickmode="array",
        tickvals=tickvals,
        ticktext=ticktext,
        row=2, col=1
    )
    fig.update_yaxes(title_text="Vertical Angle (°)", range=[V_MIN, V_MAX], row=2, col=1)

    # Layout settings
    poi_suffix = " + POI Markers" if enable_poi and poi_points else ""
    fig.update_layout(
        title_text=f"<b>{group_name} Group ({sessions_count} sessions)</b><br>{target_video} Eye Tracking Heatmap{poi_suffix}",
        height=1000,
        width=1200,
        margin=dict(t=60, b=40, l=60, r=60),
        font=dict(size=12),
        legend=dict(
            x=1.02,
            y=1,
            xanchor="left",
            yanchor="top"
        )
    )

    return fig

def create_heatmap(heat_h_norm, heat_v_norm, text_h, text_v, group_name, sessions_count):
    """Create basic heatmap (for backward compatibility)"""
    return create_heatmap_with_poi(heat_h_norm, heat_v_norm, text_h, text_v, group_name, sessions_count, [])

# Load POI data if enabled
poi_points = []
if enable_poi:
    poi_points = load_poi_data()
    print("\n=== POI Data Preview ===")
    for i, poi in enumerate(poi_points):
        print(f"  {i+1}. Time: {poi['time']}s, Horizontal: {poi['Hangle']}°, Vertical: {poi['Vangle']}°")
        print(f"      Hint: {poi['hint']}")
else:
    print("POI markers disabled")

# 1. Get session directories
session_dirs, plot_title, guidance_dirs, control_dirs, guidance_first_dirs, control_first_dirs = get_session_directories()
print(f"\nTotal found {len(session_dirs)} session directories")

if not session_dirs:
    print("Error: No session data found!")
    exit()

# 2. Calculate total duration
max_dur = 0
for sess in session_dirs:
    session_path = os.path.join(root_data_dir, sess)
    horizontal_file = os.path.join(session_path, "Horizontal.json")
    if os.path.exists(horizontal_file):
        info = json.load(open(horizontal_file))
        max_dur = max(max_dur, info.get("videoDuration", 0))

total_sec = int(max_dur) + 1
print(f"\nVideo total duration: {total_sec} seconds")

# Choose time scale interval
interval = 60 if total_sec > 120 else 10
tickvals = list(range(0, total_sec, interval))
if (total_sec - 1) not in tickvals:
    tickvals.append(total_sec - 1)
ticktext = [f"{val//60}:{val%60:02d}" for val in tickvals]

# 3. Process different modes
if mode == "all_groups" and guidance_dirs and control_dirs:
    # Process Guidance group
    heat_h_guidance, heat_v_guidance, text_h_guidance, text_v_guidance, guidance_sessions = process_group_data(guidance_dirs, "Guidance")
    
    # Process Control group
    heat_h_control, heat_v_control, text_h_control, text_v_control, control_sessions = process_group_data(control_dirs, "Control")
    
    # Generate two heatmaps
    print("\n=== Generating Guidance Group Heatmap ===")
    fig_guidance = create_heatmap_with_poi(heat_h_guidance, heat_v_guidance, text_h_guidance, text_v_guidance, "Guidance", guidance_sessions, poi_points)
    fig_guidance.show()
    
    print("\n=== Generating Control Group Heatmap ===")
    fig_control = create_heatmap_with_poi(heat_h_control, heat_v_control, text_h_control, text_v_control, "Control", control_sessions, poi_points)
    fig_control.show()

elif mode == "all_groups_enhanced" and guidance_dirs and control_dirs and guidance_first_dirs and control_first_dirs:
    # Enhanced mode: Generate 4 heatmaps
    
    # 1. Guidance group first session data
    print("\n=== Processing Guidance Group First Session Data ===")
    heat_h_guidance_first, heat_v_guidance_first, text_h_guidance_first, text_v_guidance_first, guidance_first_sessions = process_group_data(guidance_first_dirs, "Guidance First")
    fig_guidance_first = create_heatmap_with_poi(heat_h_guidance_first, heat_v_guidance_first, text_h_guidance_first, text_v_guidance_first, "Guidance First Sessions", guidance_first_sessions, poi_points)
    fig_guidance_first.show()
    
    # 2. Control group first session data
    print("\n=== Processing Control Group First Session Data ===")
    heat_h_control_first, heat_v_control_first, text_h_control_first, text_v_control_first, control_first_sessions = process_group_data(control_first_dirs, "Control First")
    fig_control_first = create_heatmap_with_poi(heat_h_control_first, heat_v_control_first, text_h_control_first, text_v_control_first, "Control First Sessions", control_first_sessions, poi_points)
    fig_control_first.show()
    
    # 3. Guidance group all data
    print("\n=== Processing Guidance Group All Data ===")
    heat_h_guidance_all, heat_v_guidance_all, text_h_guidance_all, text_v_guidance_all, guidance_all_sessions = process_group_data(guidance_dirs, "Guidance All")
    fig_guidance_all = create_heatmap_with_poi(heat_h_guidance_all, heat_v_guidance_all, text_h_guidance_all, text_v_guidance_all, "Guidance All Sessions", guidance_all_sessions, poi_points)
    fig_guidance_all.show()
    
    # 4. Control group all data
    print("\n=== Processing Control Group All Data ===")
    heat_h_control_all, heat_v_control_all, text_h_control_all, text_v_control_all, control_all_sessions = process_group_data(control_dirs, "Control All")
    fig_control_all = create_heatmap_with_poi(heat_h_control_all, heat_v_control_all, text_h_control_all, text_v_control_all, "Control All Sessions", control_all_sessions, poi_points)
    fig_control_all.show()
    
else:
    # Other mode processing logic (single user or all users first session)
    if mode == "single_user":
        target_dirs = session_dirs
        title_suffix = f"{user_name} — All Sessions"
    elif mode == "first_session_all":
        target_dirs = session_dirs
        title_suffix = "All Users — First Sessions"
    else:
        target_dirs = session_dirs
        title_suffix = plot_title
    
    # Process data (using existing logic)
    heat_h_norm, heat_v_norm, text_h, text_v, processed_count = process_group_data(target_dirs, "Combined")
    
    # Generate single heatmap
    fig = create_heatmap_with_poi(heat_h_norm, heat_v_norm, text_h, text_v, title_suffix, processed_count, poi_points)
    fig.show()

print(f"\n{'='*60}")
print("✅ All heatmaps with POI markers generated successfully!" if enable_poi else "✅ All heatmaps generated successfully!")
if enable_poi and poi_points:
    print("🔴 Red circles indicate POI points of interest")
    print("💡 Hover over red dots to view detailed information")
print(f"{'='*60}")

In [16]:
def calculate_actual_viewing_time_from_existing_data():
    """Calculate actual viewing time from existing TrailData (including jumps and re-watching)"""
    
    print("=== Calculating Actual Viewing Time from Existing Data ===")
    
    user_stats = {}
    
    for user_dir in os.listdir(root_data_dir):
        if user_dir.endswith('.meta'):
            continue
            
        user_path = os.path.join(root_data_dir, user_dir)
        if not os.path.isdir(user_path):
            continue
            
        print(f"\nProcessing user: {user_dir}")
        
        # Collect all sessions for this user
        user_sessions = []
        for session_dir in os.listdir(user_path):
            if target_video in session_dir and not session_dir.endswith('.meta'):
                session_path = os.path.join(user_path, session_dir)
                
                # Extract timestamp for sorting
                parts = session_dir.split('_')
                if len(parts) >= 3:
                    timestamp = parts[2]
                    user_sessions.append((timestamp, session_dir, session_path))
        
        # Sort by timestamp
        user_sessions.sort()
        
        if len(user_sessions) <= 1:
            print(f"  User {user_dir} has only {len(user_sessions)} session(s), skipping")
            user_stats[user_dir] = {
                'total_sessions': len(user_sessions),
                'actual_viewing_time': 0,
                'sessions_detail': []
            }
            continue
        
        # Exclude first session, calculate actual viewing time for subsequent sessions
        excluded_sessions = user_sessions[1:]
        user_actual_time = 0
        sessions_detail = []
        
        print(f"  Excluding first session: {user_sessions[0][1]}")
        print(f"  Calculating viewing time for {len(excluded_sessions)} subsequent sessions:")
        
        for timestamp, session_dir, session_path in excluded_sessions:
            session_actual_time = calculate_session_actual_time(session_path, session_dir)
            
            if session_actual_time > 0:
                user_actual_time += session_actual_time
                sessions_detail.append({
                    'session': session_dir,
                    'actual_time': session_actual_time
                })
                
                print(f"    {session_dir}: {session_actual_time:.1f}s ({session_actual_time/60:.1f}min)")
            else:
                print(f"    {session_dir}: No valid data")
        
        user_stats[user_dir] = {
            'total_sessions': len(user_sessions),
            'excluded_sessions': len(excluded_sessions),
            'actual_viewing_time': user_actual_time,
            'sessions_detail': sessions_detail
        }
        
        print(f"  User {user_dir} total actual viewing time: {user_actual_time:.1f}s ({user_actual_time/60:.1f}min)")
    
    # Output statistics (simplified - only user data excluding first sessions)
    print(f"\n{'='*60}")
    print("📊 Actual Viewing Time Statistics (Excluding First Sessions):")
    print(f"{'='*60}")
    
    total_users = len(user_stats)
    users_with_data = sum(1 for stats in user_stats.values() if stats['actual_viewing_time'] > 0)
    
    print(f"Total users: {total_users}")
    print(f"Users with viewing data: {users_with_data}")
    
    if users_with_data > 0:
        total_user_time = sum(stats['actual_viewing_time'] for stats in user_stats.values())
        avg_time = total_user_time / users_with_data
        print(f"Total user viewing time (excluding first sessions): {total_user_time:.1f}s ({total_user_time/60:.1f}min)")
        print(f"Average viewing time per user: {avg_time:.1f}s ({avg_time/60:.1f}min)")
    
    return user_stats

def calculate_session_actual_time(session_path, session_name):
    """Calculate actual viewing time for a single session (based on data recording interval)"""
    
    horizontal_file = os.path.join(session_path, "Horizontal.json")
    
    if not os.path.exists(horizontal_file):
        return 0
    
    try:
        with open(horizontal_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        points = data.get("points", [])
        if len(points) < 2:
            return 0
        
        # Calculate actual viewing time based on recordInterval
        # From code analysis: recordInterval = 0.1 seconds
        record_interval = 0.5  # Corresponding to recordInterval in code
        
        # Method 2: More precise calculation - considering time jumps
        actual_time_precise = calculate_precise_viewing_time(points, record_interval)
        
        return actual_time_precise
        
    except Exception as e:
        print(f"    Error: Unable to calculate viewing time for {session_name}: {e}")
        return 0

def calculate_precise_viewing_time(points, record_interval):
    """Precisely calculate viewing time, considering time jumps"""
    
    if len(points) < 2:
        return 0
    
    # Sort by time
    sorted_points = sorted(points, key=lambda x: x["time"])
    
    total_viewing_time = 0
    continuous_segments = []
    current_segment_start = 0
    
    # Identify continuous viewing segments
    for i in range(1, len(sorted_points)):
        prev_time = sorted_points[i-1]["time"]
        curr_time = sorted_points[i]["time"]
        time_diff = curr_time - prev_time
        
        # If time difference exceeds 2x recording interval, consider it a jump
        if time_diff > record_interval * 2:
            # End current continuous segment
            segment_length = i - current_segment_start
            segment_time = segment_length * record_interval
            continuous_segments.append({
                'start_index': current_segment_start,
                'end_index': i-1,
                'length': segment_length,
                'time': segment_time,
                'start_video_time': sorted_points[current_segment_start]["time"],
                'end_video_time': sorted_points[i-1]["time"]
            })
            total_viewing_time += segment_time
            
            # Start new continuous segment
            current_segment_start = i-1
    
    # Handle last continuous segment
    if current_segment_start < len(sorted_points) - 1:
        segment_length = len(sorted_points) - current_segment_start
        segment_time = segment_length * record_interval
        continuous_segments.append({
            'start_index': current_segment_start,
            'end_index': len(sorted_points)-1,
            'length': segment_length,
            'time': segment_time,
            'start_video_time': sorted_points[current_segment_start]["time"],
            'end_video_time': sorted_points[-1]["time"]
        })
        total_viewing_time += segment_time
    
    return total_viewing_time


# Execute calculation
detailed_stats = calculate_actual_viewing_time_from_existing_data()

=== Calculating Actual Viewing Time from Existing Data ===

Processing user: Control_1
  Excluding first session: Control_1_20250710_214426_20180717_monkey cinema_focal manny_injected_360
  Calculating viewing time for 4 subsequent sessions:
    Control_1_20250710_214828_20180717_monkey cinema_focal manny_injected_360: 187.0s (3.1min)
    Control_1_20250710_214948_20180717_monkey cinema_focal manny_injected_360: 72.5s (1.2min)
    Control_1_20250710_215203_20180717_monkey cinema_focal manny_injected_360: 128.0s (2.1min)
    Control_1_20250710_215300_20180717_monkey cinema_focal manny_injected_360: 48.5s (0.8min)
  User Control_1 total actual viewing time: 436.0s (7.3min)

Processing user: Control_2
  Excluding first session: Control_2_20250709_145020_20180717_monkey cinema_focal manny_injected_360
  Calculating viewing time for 2 subsequent sessions:
    Control_2_20250709_145414_20180717_monkey cinema_focal manny_injected_360: 187.0s (3.1min)
    Control_2_20250709_145735_20180717_mon